In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
from datasets import load_dataset
data_files = {"train": "data.csv"}
dataset = load_dataset("scherrmann/financial_phrasebank_75agree_german")
german_df = pd.DataFrame(dataset['train'])

In [26]:
import kagglehub
import pandas as pd

path = kagglehub.dataset_download("arcticgiant/french-financial-news")
french_df = pd.read_csv(f'{path}/FrenchNews.csv')

In [ ]:
german_df.isnull().sum()

sentence    0
label       0
dtype: int64

In [ ]:
# sentiment_map = {'negative': 0, 'neutral': 1, 'positive': 2}
sentiment_map = {0: 'negative', 1: 'neutral', 2: 'positive'}
german_df['Sentiment'] = df['label'].map(sentiment_map)
german_df.rename(columns={'sentence': 'Sentence'}, inplace=True)

In [ ]:
german_df['Sentiment'].value_counts()

Sentiment
neutral     1717
positive     710
negative     336
Name: count, dtype: int64

In [ ]:
french_df['sentiment_title'] = french_df['Sentiment Vader Title'].apply(lambda x: 'positive' if x > 0.05 else ('negative' if x < -0.05 else 'neutral'))
french_df['sentiment_text'] = french_df['Sentiment Vader Text'].apply(lambda x: 'positive' if x > 0.05 else ('negative' if x < -0.05 else 'neutral'))
french_df['sentiment_url'] = french_df['Sentiment Vader TextURL'].apply(lambda x: 'positive' if x > 0.05 else ('negative' if x < -0.05 else 'neutral'))
french_df_processed = french_df.drop(columns=['Sentiment Vader Title', 'Sentiment Vader Text', 'Sentiment Vader TextURL', 'Numero news', 'Numero page', 'Numero', 'Agency'])
french_df_processed = french_df_processed.rename(columns={"Titre": "title", "Contenu": "text", "URL": "url"})
french_df_processed = french_df_processed[['title','sentiment_title']]
french_df_processed = french_df_processed.rename(columns={"title": "Sentence", "sentiment_title": "Sentiment"})
french_df_processed['Sentiment'].value_counts()

Sentiment
neutral     18507
negative    13188
positive     9848
Name: count, dtype: int64

In [32]:
df = pd.concat([german_df, french_df_processed], ignore_index=True)
df['Sentiment'].value_counts()

NameError: name 'german_df' is not defined

In [22]:
train, eval = train_test_split(df, test_size=0.2, random_state=42, stratify=df['Sentiment'])
train.shape, eval.shape

((2210, 3), (553, 3))

In [23]:
train.to_csv("data/train_subset.csv", index=False)
eval.to_csv("data/eval_subset.csv", index=False)